In [1]:
import numpy as np
import pandas as pd

In [6]:
file_path = '/Volumes/YCL/avazu_dataset/train.csv'
data = pd.read_csv(file_path, chunksize=10000, iterator=True)
data.get_chunk(5)

,order_detail_id,order_id,order_total_num,order_amount,order_total_payment,order_total_discount,order_pay_time,order_status,order_count,is_customer_rate,...,customer_gender,member_status,is_member_actived,goods_id,goods_class_id,goods_price,goods_status,goods_has_discount,goods_list_time,goods_delist_time
0,1000000,1000000,1.0,239.9,96.9,0.0,2012-11-01 00:10:56,6,1.0,0.0,...,NaN,NaN,NaN,998,998,54.909289,1.0,0.0,2014-10-25 11:08:07,2014-11-01 11:08:07
1,1001530,1001327,2.0,288.0,96.9,0.0,2013-08-31 23:14:42,6,2.0,0.0,...,NaN,NaN,NaN,1953,1953,45.961352,0.0,1.0,2013-08-28 17:27:50,2013-09-01 00:38:17
2,1001531,1001327,2.0,288.0,96.9,0.0,2013-08-31 23:14:42,6,2.0,0.0,...,NaN,NaN,NaN,1083,1083,53.035439,1.0,0.0,2014-10-29 18:21:05,2014-11-05 18:21:05
3,1001532,1001328,3.0,180.0,89.7,0.0,2013-08-31 22:06:35,6,1.0,0.0,...,NaN,NaN,NaN,1013,1013,46.046917,1.0,1.0,2014-10-25 11:00:00,2014-11-01 11:00:00
4,1001533,1001329,1.0,159.9,65.9,0.0,2013-08-31 21:33:36,6,1.0,0.0,...,NaN,NaN,NaN,1628,1628,50.722161,1.0,0.0,2014-10-23 15:35:33,2014-10-30 15:35:33


In [8]:
data.columns

AttributeError: 'TextFileReader' object has no attribute 'columns'

In [3]:
print(data.columns)
print(data['click'].value_counts())

Index(['Unnamed: 0', 'id', 'click', 'hour', 'C1', 'banner_pos', 'site_id',
       'site_domain', 'site_category', 'app_id', 'app_domain', 'app_category',
       'device_id', 'device_ip', 'device_model', 'device_type',
       'device_conn_type', 'C14', 'C15', 'C16', 'C17', 'C18', 'C19', 'C20',
       'C21'],
      dtype='object')
0    8294
1    1706
Name: click, dtype: int64


In [4]:
data = data.drop(['id'], axis=1)
print(data.columns)

Index(['Unnamed: 0', 'click', 'hour', 'C1', 'banner_pos', 'site_id',
       'site_domain', 'site_category', 'app_id', 'app_domain', 'app_category',
       'device_id', 'device_ip', 'device_model', 'device_type',
       'device_conn_type', 'C14', 'C15', 'C16', 'C17', 'C18', 'C19', 'C20',
       'C21'],
      dtype='object')


In [5]:
for fea in data.drop('click', axis=1).columns:
    data[fea] = pd.factorize(data[fea])[0]

from sklearn.model_selection import train_test_split
train, test = train_test_split(data, test_size=0.2)

In [7]:
import lightgbm as lgb

In [8]:
clf = lgb.LGBMClassifier(is_unbalanced=False, slient=False)
cal = list(train.drop('click', axis=1).columns)
print(cal)

['Unnamed: 0', 'hour', 'C1', 'banner_pos', 'site_id', 'site_domain', 'site_category', 'app_id', 'app_domain', 'app_category', 'device_id', 'device_ip', 'device_model', 'device_type', 'device_conn_type', 'C14', 'C15', 'C16', 'C17', 'C18', 'C19', 'C20', 'C21']


In [10]:
train['unnamed'] = train.pop('Unnamed: 0')
cal = list(train.drop('click', axis=1).columns)
print(cal)
clf.fit(train.drop('click', axis=1), train['click'], categorical_feature=cal, verbose=5)
predict = clf.predict_proba(test.drop('click', axis=1))[:, 1]

def celoss(target, predict):
    target = np.array(target)
    predict = np.array(predict)
    return -(target*np.log(predict)+(1-target)*np.log(1-predict)).mean()

print(celoss(test['click'], predict))

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/lightgbm/basic.py:1295: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is ['C1', 'C14', 'C15', 'C16', 'C17', 'C18', 'C19', 'C20', 'C21', 'app_category', 'app_domain', 'app_id', 'banner_pos', 'device_conn_type', 'device_id', 'device_ip', 'device_model', 'device_type', 'hour', 'site_category', 'site_domain', 'site_id', 'unnamed']
  'New categorical_feature is {}'.format(sorted(list(categorical_feature))))


['hour', 'C1', 'banner_pos', 'site_id', 'site_domain', 'site_category', 'app_id', 'app_domain', 'app_category', 'device_id', 'device_ip', 'device_model', 'device_type', 'device_conn_type', 'C14', 'C15', 'C16', 'C17', 'C18', 'C19', 'C20', 'C21', 'unnamed']
0.551570857693822
